In [8]:
#Load Dependencies
import datetime as dt
from datetime import datetime
import numpy as np
import pandas as pd
from uszipcode import SearchEngine

In [2]:
#read in csv for data from December 2020
data = pd.read_csv('JC-202012-citibike-tripdata.csv')
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,146,2020-12-01 00:02:50.1450,2020-12-01 00:05:16.1940,3202,Newport PATH,40.727224,-74.033759,3199,Newport Pkwy,40.728745,-74.032108,42308,Subscriber,1989,2
1,572,2020-12-01 00:11:57.3910,2020-12-01 00:21:30.2510,3640,Journal Square,40.733670,-74.062500,3280,Astor Place,40.719282,-74.071262,18568,Subscriber,1997,2
2,387,2020-12-01 00:14:49.3610,2020-12-01 00:21:16.8730,3640,Journal Square,40.733670,-74.062500,3194,McGinley Square,40.725340,-74.067622,44543,Subscriber,1960,1
3,188,2020-12-01 00:45:06.3680,2020-12-01 00:48:14.4280,3186,Grove St PATH,40.719586,-74.043117,3270,Jersey & 6th St,40.725289,-74.045572,43098,Subscriber,1998,1
4,594,2020-12-01 01:17:17.0110,2020-12-01 01:27:11.9400,3212,Christ Hospital,40.734786,-74.050444,3209,Brunswick St,40.724176,-74.050656,44723,Subscriber,1988,1


In [9]:
#grab all stations in start & end station clumns

start_stations = data[['start station id','start station latitude', 'start station longitude']]
start_stations = start_stations.rename(columns={'start station id': 'Station_ID','start station latitude': 'Latitude', 'start station longitude': 'Longitude'})
end_stations = data[['end station id','end station latitude', 'end station longitude']]
end_stations = end_stations.rename(columns={'end station id': 'Station_ID','end station latitude': 'Latitude', 'end station longitude': 'Longitude'})

In [10]:
#combine stations listed in both start & end station columns
combined_stations = pd.concat([start_stations, end_stations])
combined_stations

,Station_ID,Latitude,Longitude
0,3202,40.727224,-74.033759
1,3640,40.733670,-74.062500
2,3640,40.733670,-74.062500
3,3186,40.719586,-74.043117
4,3212,40.734786,-74.050444
...,...,...,...
11689,3199,40.728745,-74.032108
11690,3195,40.730897,-74.063913
11691,3270,40.725289,-74.045572
11692,3199,40.728745,-74.032108


In [14]:
#find all unique station numbers
all_stations = combined_stations.drop_duplicates(subset='Station_ID')
all_stations.head()

,Station_ID,Latitude,Longitude
0,3202,40.727224,-74.033759
1,3640,40.733670,-74.062500
3,3186,40.719586,-74.043117
4,3212,40.734786,-74.050444
5,3194,40.725340,-74.067622
6,3184,40.714145,-74.033552
9,3199,40.728745,-74.032108
11,3678,40.725726,-74.071959
17,3679,40.722104,-74.071455
18,3206,40.731169,-74.057574


In [18]:
#test using new function
search = SearchEngine()
test_zip = search.by_coordinates(all_stations['Latitude'][0], all_stations['Longitude'][0], returns=1)[0].to_dict()['zipcode']
test_zip

'07310'

In [44]:
#get zip codes for all stations in data set
station_details = []
for index, row in all_stations.iterrows():
    zipcode = search.by_coordinates(row['Latitude'], row['Longitude'], returns=1)[0].to_dict()['zipcode']
    station_details.append([int(row['Station_ID']), zipcode])
Station_Zips = pd.DataFrame(station_details, columns = ['Station ID', 'Zip Code'])
Station_Zips.head

<bound method NDFrame.head of     Station ID Zip Code
0         3202    07310
1         3640    07306
2         3186    07302
3         3212    07310
4         3194    07302
5         3184    07311
6         3199    07310
7         3678    07306
8         3679    07304
9         3206    07302
10        3195    07306
11        3270    07302
12        3279    07302
13        3203    07310
14        3187    07311
15        3681    07311
16        3677    07306
17        3267    07311
18        3268    07304
19        3278    07302
20        3269    07302
21        3276    07302
22        3209    07302
23        3213    07302
24        3205    07302
25        3214    07311
26        3275    07311
27        3638    07311
28        3201    07306
29        3192    07302
30        3193    07304
31        3273    07302
32        3280    07304
33        3792    07311
34        3272    07302
35        3639    07311
36        3211    07302
37        3210    07307
38        3225    07302
39        

In [55]:
#add zip codes for start stations
new_df = pd.merge(data, Station_Zips,how = 'left', right_on = 'Station ID', left_on = 'start station id', suffixes=("_x", "_y"))
new_df = new_df.rename(columns={'Zip Code': 'Start Station Zip'})
new_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Station ID,Start Station Zip
0,146,2020-12-01 00:02:50.1450,2020-12-01 00:05:16.1940,3202,Newport PATH,40.727224,-74.033759,3199,Newport Pkwy,40.728745,-74.032108,42308,Subscriber,1989,2,3202,07310
1,572,2020-12-01 00:11:57.3910,2020-12-01 00:21:30.2510,3640,Journal Square,40.733670,-74.062500,3280,Astor Place,40.719282,-74.071262,18568,Subscriber,1997,2,3640,07306
2,387,2020-12-01 00:14:49.3610,2020-12-01 00:21:16.8730,3640,Journal Square,40.733670,-74.062500,3194,McGinley Square,40.725340,-74.067622,44543,Subscriber,1960,1,3640,07306
3,188,2020-12-01 00:45:06.3680,2020-12-01 00:48:14.4280,3186,Grove St PATH,40.719586,-74.043117,3270,Jersey & 6th St,40.725289,-74.045572,43098,Subscriber,1998,1,3186,07302
4,594,2020-12-01 01:17:17.0110,2020-12-01 01:27:11.9400,3212,Christ Hospital,40.734786,-74.050444,3209,Brunswick St,40.724176,-74.050656,44723,Subscriber,1988,1,3212,07310


In [58]:
#Add zip codes for end stations
merged_df = pd.merge(new_df, Station_Zips,how = 'left', right_on = 'Station ID', left_on = 'end station id', suffixes=("_x", "_y"))
merged_df = merged_df.rename(columns={'Zip Code': 'End Station Zip'})
merged_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Station ID_x,Start Station Zip,Station ID_y,End Station Zip
0,146,2020-12-01 00:02:50.1450,2020-12-01 00:05:16.1940,3202,Newport PATH,40.727224,-74.033759,3199,Newport Pkwy,40.728745,-74.032108,42308,Subscriber,1989,2,3202,07310,3199,07310
1,572,2020-12-01 00:11:57.3910,2020-12-01 00:21:30.2510,3640,Journal Square,40.733670,-74.062500,3280,Astor Place,40.719282,-74.071262,18568,Subscriber,1997,2,3640,07306,3280,07304
2,387,2020-12-01 00:14:49.3610,2020-12-01 00:21:16.8730,3640,Journal Square,40.733670,-74.062500,3194,McGinley Square,40.725340,-74.067622,44543,Subscriber,1960,1,3640,07306,3194,07302
3,188,2020-12-01 00:45:06.3680,2020-12-01 00:48:14.4280,3186,Grove St PATH,40.719586,-74.043117,3270,Jersey & 6th St,40.725289,-74.045572,43098,Subscriber,1998,1,3186,07302,3270,07302
4,594,2020-12-01 01:17:17.0110,2020-12-01 01:27:11.9400,3212,Christ Hospital,40.734786,-74.050444,3209,Brunswick St,40.724176,-74.050656,44723,Subscriber,1988,1,3212,07310,3209,07302


In [59]:
#remove extra columns
final_df = merged_df.drop(columns=['Station ID_y', 'Station ID_x'])
final_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Start Station Zip,End Station Zip
0,146,2020-12-01 00:02:50.1450,2020-12-01 00:05:16.1940,3202,Newport PATH,40.727224,-74.033759,3199,Newport Pkwy,40.728745,-74.032108,42308,Subscriber,1989,2,07310,07310
1,572,2020-12-01 00:11:57.3910,2020-12-01 00:21:30.2510,3640,Journal Square,40.733670,-74.062500,3280,Astor Place,40.719282,-74.071262,18568,Subscriber,1997,2,07306,07304
2,387,2020-12-01 00:14:49.3610,2020-12-01 00:21:16.8730,3640,Journal Square,40.733670,-74.062500,3194,McGinley Square,40.725340,-74.067622,44543,Subscriber,1960,1,07306,07302
3,188,2020-12-01 00:45:06.3680,2020-12-01 00:48:14.4280,3186,Grove St PATH,40.719586,-74.043117,3270,Jersey & 6th St,40.725289,-74.045572,43098,Subscriber,1998,1,07302,07302
4,594,2020-12-01 01:17:17.0110,2020-12-01 01:27:11.9400,3212,Christ Hospital,40.734786,-74.050444,3209,Brunswick St,40.724176,-74.050656,44723,Subscriber,1988,1,07310,07302


In [60]:
#Convert gender from numeric values to string values
genders = {0:"Unknown", 1 : "Male", 2 :"Female"}
final_df['gender'] = [genders[item] for item in final_df['gender']]
final_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Start Station Zip,End Station Zip
0,146,2020-12-01 00:02:50.1450,2020-12-01 00:05:16.1940,3202,Newport PATH,40.727224,-74.033759,3199,Newport Pkwy,40.728745,-74.032108,42308,Subscriber,1989,Female,07310,07310
1,572,2020-12-01 00:11:57.3910,2020-12-01 00:21:30.2510,3640,Journal Square,40.733670,-74.062500,3280,Astor Place,40.719282,-74.071262,18568,Subscriber,1997,Female,07306,07304
2,387,2020-12-01 00:14:49.3610,2020-12-01 00:21:16.8730,3640,Journal Square,40.733670,-74.062500,3194,McGinley Square,40.725340,-74.067622,44543,Subscriber,1960,Male,07306,07302
3,188,2020-12-01 00:45:06.3680,2020-12-01 00:48:14.4280,3186,Grove St PATH,40.719586,-74.043117,3270,Jersey & 6th St,40.725289,-74.045572,43098,Subscriber,1998,Male,07302,07302
4,594,2020-12-01 01:17:17.0110,2020-12-01 01:27:11.9400,3212,Christ Hospital,40.734786,-74.050444,3209,Brunswick St,40.724176,-74.050656,44723,Subscriber,1988,Male,07310,07302


In [61]:
#Export data to csv for import to Tableau
final_df.to_csv('New_CitiBike_TripData_December2020.csv', index = False)

In [62]:
#Repeat process above to format data from Jan 2019
Jan_2019_Data = pd.read_csv('201901-citibike-tripdata.csv')
Jan_2019_Data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,320,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,3160.0,Central Park West & W 76 St,40.778968,-73.973747,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1
1,316,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1
2,591,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154.0,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1
3,2719,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1
4,303,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1


In [63]:
##Jan 2019 Data

start_stations_2019 = Jan_2019_Data[['start station id','start station latitude', 'start station longitude']]
start_stations_2019 = start_stations_2019.rename(columns={'start station id': 'Station_ID','start station latitude': 'Latitude', 'start station longitude': 'Longitude'})
end_stations_2019 = Jan_2019_Data[['end station id','end station latitude', 'end station longitude']]
end_stations_2019 = end_stations_2019.rename(columns={'end station id': 'Station_ID','end station latitude': 'Latitude', 'end station longitude': 'Longitude'})

In [66]:
combined_stations_2019 = pd.concat([start_stations_2019, end_stations_2019])
combined_stations_2019.head()

,Station_ID,Latitude,Longitude
0,3160.0,40.778968,-73.973747
1,519.0,40.751873,-73.977706
2,3171.0,40.785247,-73.976673
3,504.0,40.732219,-73.981656
4,229.0,40.727434,-73.993790


In [67]:
#Grab all stations & their Lat/Long
all_stations_2019 = combined_stations_2019.drop_duplicates(subset='Station_ID')
all_stations_2019.head()

,Station_ID,Latitude,Longitude
0,3160.0,40.778968,-73.973747
1,519.0,40.751873,-73.977706
2,3171.0,40.785247,-73.976673
3,504.0,40.732219,-73.981656
4,229.0,40.727434,-73.993790


In [70]:
#Use station Lat/Lng to retrieve zip codes
station_details_2019 = []
for index, row in all_stations_2019.iterrows():
    zipcode = search.by_coordinates(row['Latitude'], row['Longitude'], returns=1)[0].to_dict()['zipcode']
    station_details_2019.append([row['Station_ID'], zipcode])
Station_Zips_2019 = pd.DataFrame(station_details_2019, columns = ['Station ID', 'Zip Code'])
Station_Zips_2019.head()

,Station ID,Zip Code
0,3160.0,10023
1,519.0,10110
2,3171.0,10023
3,504.0,10009
4,229.0,10012


In [71]:
#Add start & end station zip codes to full dataframe
new_df_2019 = pd.merge(Jan_2019_Data, Station_Zips_2019,how = 'left', right_on = 'Station ID', left_on = 'start station id', suffixes=("_x", "_y"))
new_df_2019 = new_df_2019.rename(columns={'Zip Code': 'Start Station Zip'})
new_df_2019.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Station ID,Start Station Zip
0,320,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,3160.0,Central Park West & W 76 St,40.778968,-73.973747,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1,3160.0,10023
1,316,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1,519.0,10110
2,591,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154.0,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1,3171.0,10023
3,2719,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1,504.0,10009
4,303,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1,229.0,10012


In [72]:
merged_df_2019 = pd.merge(new_df_2019, Station_Zips_2019,how = 'left', right_on = 'Station ID', left_on = 'end station id', suffixes=("_x", "_y"))
merged_df_2019 = merged_df_2019.rename(columns={'Zip Code': 'End Station Zip'})
merged_df_2019.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Station ID_x,Start Station Zip,Station ID_y,End Station Zip
0,320,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,3160.0,Central Park West & W 76 St,40.778968,-73.973747,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1,3160.0,10023,3283.0,10162
1,316,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1,519.0,10110,518.0,10017
2,591,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154.0,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1,3171.0,10023,3154.0,10021
3,2719,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1,504.0,10009,3709.0,10011
4,303,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1,229.0,10012,503.0,10003


In [73]:
#Remove un-needed columns
final_df_2019 = merged_df_2019.drop(columns=['Station ID_y', 'Station ID_x'])
final_df_2019.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Start Station Zip,End Station Zip
0,320,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,3160.0,Central Park West & W 76 St,40.778968,-73.973747,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1,10023,10162
1,316,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1,10110,10017
2,591,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154.0,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1,10023,10021
3,2719,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1,10009,10011
4,303,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1,10012,10003


In [74]:
genders = {0:"Unknown", 1 : "Male", 2 :"Female"}
final_df_2019['gender'] = [genders[item] for item in final_df_2019['gender']]
final_df_2019.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Start Station Zip,End Station Zip
0,320,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,3160.0,Central Park West & W 76 St,40.778968,-73.973747,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,Male,10023,10162
1,316,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,Male,10110,10017
2,591,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154.0,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,Male,10023,10021
3,2719,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,Male,10009,10011
4,303,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,Male,10012,10003


In [75]:
final_df_2019.to_csv('CitiBike_TripData_January2019.csv', index = False)